In [ ]:
 !pip install -q -U bitsandbytes
 !pip install -q -U transformers
 !pip install -q -U xformers
 !pip install -q -U peft
 !pip install -q -U accelerate
 !pip install -q -U datasets
 !pip install -q -U trl
 !pip install -q -U einops
 !pip install -q -U nvidia-ml-py3
 !pip install -q -U huggingface_hub

# Importing required libraries

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from pynvml import *
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import time, torch
from peft import AutoPeftModelForCausalLM
from peft import LoraConfig,PeftModel,AutoPeftModelForCausalLM

# Loading the dataset

In [3]:
 dataset = load_dataset("b-mc2/sql-create-context")
 dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

# Formatting the dataset

In [5]:
def create_prompt(sample):
      system_prompt_template = """<s>
            Below is an instruction that describes a task.Write a response that appropriately completes the request.
            ### Instruction :<<user_question>>
            ### Database Schema:
            <<database_schema>>
            ### Response:
            <<user_response>>
            </s>
            """
      user_message = sample['question']
      user_response = sample['answer']
      database_schema = sample['context']
      prompt_template = system_prompt_template.replace("<<user_question>>",f"{user_message}").replace("<<user_response>>",f"{user_response}").replace("<<database_schema>>",f"{database_schema} ")

      return {"inputs":prompt_template}


instruct_tune_dataset = dataset.map(create_prompt)
print(instruct_tune_dataset)


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context', 'inputs'],
        num_rows: 78577
    })
})


In [10]:
print(instruct_tune_dataset['train']['inputs'][0])

<s>
            Below is an instruction that describes a task.Write a response that appropriately completes the request.
            ### Instruction :How many heads of the departments are older than 56 ?
            ### Database Schema:
            CREATE TABLE head (age INTEGER) 
            ### Response:
            SELECT COUNT(*) FROM head WHERE age > 56
            </s>
            


In [14]:
base_model_id = "microsoft/Phi-3-mini-4k-instruct"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id  , use_fast=True)
#Load the model with fp16
model =  AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True, torch_dtype=torch.float16, device_map={"": 0})
print(print_gpu_utilization())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU memory occupied: 8159 MB.
None


In [24]:
 prompt = [
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        List all the cities in a decreasing order of each city's stations' highest latitude.
        Database Schema:
        CREATE TABLE station (city VARCHAR, lat INTEGER)
        ### Response:
        SELECT city, lat FROM station ORDER BY lat DESC;
        """,
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        'What are the positions with both players having more than 20 points and less than 10 points and are in Top 10 ranking
        Database Schema:
        CREATE TABLE player (POSITION VARCHAR, Points INTEGER, Ranking INTEGER)
        ### Response:
        SELECT POSITION, Points, Ranking
        FROM player
        WHERE Points > 20 AND Points < 10 AND Ranking IN (1,2,3,4,5,6,7,8,9,10)
        """,
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        Find the first name of the band mate that has performed in most songs.
        Database Schema:
        CREATE TABLE Songs (SongId VARCHAR); CREATE TABLE Band (firstname VARCHAR, id VARCHAR); CREATE TABLE Performance (bandmate VARCHAR)
        ### Response:
        SELECT b.firstname
        FROM Band b
        JOIN Performance p ON b.id = p.bandmate
        GROUP BY b.firstname
        ORDER BY COUNT(*) DESC
        LIMIT 1;
        """
    ]

# Initialize variables
duration = 0.0
total_length = 0

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500, no_repeat_ngram_size=10, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Actual:-----------------")
  print(prompt[i])
  print()
  print(f"Generated:-------------{total_length}")
  x=tokenizer.decode(output, skip_special_tokens=False)
  print(x[len(prompt[i]):])
  total_length = 0

Actual:-----------------

       Below is an instruction that describes a task. Write a response that appropriately completes the request.
       ### Instruction :
       List all the cities in a decreasing order of each city's stations' highest latitude.
       Database Schema:
       CREATE TABLE station (city VARCHAR, lat INTEGER)
       ### Response:
       SELECT city, lat FROM station ORDER BY lat DESC;
       

Generated:-------------500

       ### Instruction :
       Given the database schema below, write a SQL query to find the top 3 cities with the highest average temperature in the month of July, but only include cities that have more than 10 stations. Additionally, the query should exclude cities where the highest temperature recorded in July is below 30 degrees Celsius.
       Database Schema:
       TABLE stations (id INTEGER, city VARCHAR, country VARCHAR, latitude FLOAT, longitude FLOAT)
       TABLE readings (station_id INTEGER, date DATE, temperature FLOAT)
       #